# Researching Importing Data to Arkouda

Investigation of the support for importing data to Arkouda. Sources include
- Parquet Written from Pandas
- HDF5 Written from Pandas

## Data Types
- HDF5
- Parquet

## Important Notes
- Requires tables be installed, `pip install tables`

In [2]:
import arkouda as ak
import pandas as pd
import numpy as np

import h5py
import pickle

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2022.05.09+14.gd0b5bef2.dirty


In [3]:
ak.connect()

connected to arkouda server tcp://*:5555


## Create Pandas DataFrame

In [4]:
df = pd.DataFrame(data = {
    'Random_A': np.random.randint(0, 5, 5),
    'Random_B': np.random.randint(0, 5, 5),
    'Random_C': np.random.randint(0, 5, 5)
}, index = np.arange(5))

df

,Random_A,Random_B,Random_C
0,2,2,1
1,1,1,4
2,3,4,3
3,0,4,4
4,2,3,3


## Write to HDF5
1) Delete the file if it already exists
2) Write the Pandas DataFrame out to an HDF5 file. Verify file exists
3) Verify file Exists.
4) Verify file can be read back to pandas

In [5]:
import os
out_file = '/Users/ethandebandi/Documents/save_load_testing/pandas.hdf5'

# delete the file if it exists
if os.path.isfile(out_file):
    os.remove(out_file)

#write the file
df.to_hdf(out_file, key="data", format="table", mode="w")

if os.path.isfile(out_file):
    # load in default mode 'r'
    rdf = pd.read_hdf(out_file, key="data")
else:
    raise FileExistsError(f"Unable to find HDF5 file, {out_file}")

# display the read file
rdf

,Random_A,Random_B,Random_C
0,2,2,1
1,1,1,4
2,3,4,3
3,0,4,4
4,2,3,3


## Pandas Formatting Variations
With Pandas we can save with numerous configurations. Assuming that we will need to be able to support at least some variation, I am demonstrating some of the major differences below. All are stored with a similar root level, but there is a great deal of variance from there.

### Table Format no data_columns
This saves each row as a tuple of the form `(index, [row_data])`. This format is very tricky to read when not in Python (ie using Chapel). 
`df.to_hdf(out_file, key="data", format="table", mode="w")`

In [19]:
# Format resulting from df.to_hdf(out_file, key="data", format="table", mode="w")
f = h5py.File('/Users/ethandebandi/Documents/save_load_testing/pandas.hdf5', 'r')
k = list(f.keys())
print(f"File Keys:\n\t{k}")

data = f['data']
dk = list(data.keys())
print(f"In this format data will always be under 'table':\n\t{dk}")

dset = data['table']
fields = list(dset.dtype.fields.keys())
print(f"Fields stored in the hdf5 table:\n\t{fields}")
print(f"Table Data stored as list of tuples.\n\t{dset[:]}")

cols_pick = dset.attrs['values_block_0_kind']
cols = pickle.loads(cols_pick)
print(f"Columns are stored in 'values_block_0_kind attribute and pickled.\n\t{cols}")

File Keys:
	['data']
In this format data will always be under 'table':
	['_i_table', 'table']
Fields stored in the hdf5 table:
	['index', 'values_block_0']
Table Data stored as list of tuples.
	[(0, [4, 3, 4]) (1, [2, 3, 2]) (2, [3, 1, 0]) (3, [3, 1, 4])
 (4, [0, 0, 4])]
Columns are stored in 'values_block_0_kind attribute and pickled.
	['Random_A', 'Random_B', 'Random_C']


### Table Format With data columns
In this format, data is still stored as a table, but in individual fields. This format is by far the easiest to interact with.
Please note: data_columns does not have to be Boolean, it can be a list of columns to use. THis is an oversimplification for display purposes.
`df.to_hdf(out_file, key="data", format="table", mode="w", data_columns=True)`

In [31]:
f = h5py.File('/Users/ethandebandi/Documents/save_load_testing/pandas_test.hdf5', 'r')
k = list(f.keys())
print(f"File Keys:\n\t{k}")

data = f['data']
dk = list(data.keys())
print(f"In this format data will always be under 'table':\n\t{dk}")

dset = data['table']
fields = list(dset.dtype.fields.keys())
print(f"Each column now is formated as its own field.\n\t{fields}")
print(f"Table Data stored as list of tuples.\n\t{dset[:]}")

ra = dset['Random_A']
print(f"Fields can be directly accessed with this method using dataset['field_name'].\n\tRandom_A: {ra}")

File Keys:
	['data']
In this format data will always be under 'table':
	['_i_table', 'table']
Each column now is formated as its own field.
	['index', 'Random_A', 'Random_B', 'Random_C']
Table Data stored as list of tuples.
	[(0, 3, 4, 1) (1, 4, 2, 2) (2, 3, 1, 0) (3, 4, 0, 1) (4, 3, 2, 3)]
Fields can be directly accessed with this method using dataset['field_name'].
	Random_A: [3 4 3 4 3]


### Fixed Format with Data Columns
Fixed formatting is not designed for querying but has faster read/write speeds. Reviewing the format, this should be the easiest  to access.

`df.to_hdf(out_file, key="data", format="fixed", mode="w", data_columns=True)`

In [21]:
f = h5py.File('/Users/ethandebandi/Documents/save_load_testing/pandas_fixed.hdf5', 'r')
k = list(f.keys())
print(f"File Keys:\n\t{k}")

data = f['data']
dk = list(data.keys())
print(f"In this format data will always be under 'block0_values':\n\t{dk}")

cols = list(data['block0_items'])
print(f"The column names are always stored under 'block0_items':\n\t{cols}")

dset = list(data['block0_values'])
print(f"Data is stored as lists of ndarrays:\n\t{dset}")

print(list(data['axis1']))

File Keys:
	['data']
In this format data will always be under 'block0_values':
	['axis0', 'axis1', 'block0_items', 'block0_values']
The column names are always stored under 'block0_items':
	[b'Random_A', b'Random_B', b'Random_C']
Data is stored as lists of ndarrays:
	[array([2, 4, 3]), array([4, 0, 3]), array([1, 3, 2]), array([0, 1, 2]), array([2, 3, 0])]
[0, 1, 2, 3, 4]


## Import HDF5 into Arkouda
Investigation of how to read into arkouda properly, ie import from Pandas

## Additional Option
We know that we can read the .hdf5 with pandas... We could read in via pandas and use our save code to create a version arkouda can read. If both can read Arkouda's version, we can remove the pandas specific one.

### ISSUES
- Confirmed what Kyle had listed in #953, columns names are encoded in an odd fashion. In order to import from pandas, we will need to find a work around for this. Pickling should not be an issue if we are running Python 3+.
- Pickling presents the biggest issue with floats. we should be ok if we are assuming column names should not be floats.
- Objects pickled in Python3 cannot be unpickled by Python2
- Specify lower protocol for better compatibility.
- PyTables uses HIGHESTPROTOCOL by default. However, can patch by
```python
import pickle
pickle.HIGHEST_PROTOCOL = 4
import pandas

df.to_hdf(file, key)
```

### Import Methods
- Read with h5py, table (pytable), or pandas. Then, we can read the information out and into arkouda. Example below utilizes h5py.

In [16]:
# load the dataset and set attrs we need to access
f = h5py.File('/Users/ethandebandi/Documents/save_load_testing/pandas.hdf5', 'r')
k = list(f.keys())
col_name_attr =  'values_block_0_kind'
nrows_attr = 'NROWS'
dset = f['data/table']

print(dset[:])
print(dset.dtype)

# access the column names
pickled_cols = dset.attrs[col_name_attr]
cols = pickle.loads(pickled_cols)

# Create dictionary to load to arkouda
df_def = {}
for i in range(len(cols)):
    df_def[cols[i]] = ak.array(dset[:]['values_block_0'][:,i].tolist())
    
akdf = ak.DataFrame(df_def)
print(akdf.__repr__())

[(0, [4, 3, 4]) (1, [2, 3, 2]) (2, [3, 1, 0]) (3, [3, 1, 4])
 (4, [0, 0, 4])]
[('index', '<i8'), ('values_block_0', '<i8', (3,))]


RuntimeError: client is not connected to a server

## Export hdf5 file to Pandas
Arkouda HDF5 files load into pandas.

**This Example does not glue together files for multi-locale**

In [7]:
ak_outfile = "/Users/ethandebandi/Documents/save_load_testing/ak_test.hdf5"
pd_read = '/Users/ethandebandi/Documents/save_load_testing/ak_test_LOCALE0000.hdf5'

# delete the file if it exists
if os.path.isfile(pd_read):
    os.remove(pd_read)

# Write the file out using arkouda
akdf.save_table(ak_outfile, file_format='HDF5')

# Open the file with pandas
f = h5py.File(pd_read, 'r')
cols = list(f.keys())
df_def = {}
for c in cols:
    if c == "_arkouda_metadata":
        continue
    df_def[c] = f[c][:]

df = pd.DataFrame(df_def)
df
    

,Random_A,Random_B,Random_C
0,2,2,1
1,1,1,4
2,3,4,3
3,0,4,4
4,2,3,3


## Import Parquet to Arkouda

### Potential Issues
- engine used to save from pandas are fastparquet. We want to use pyarrow

In [8]:
out_file = '/Users/ethandebandi/Documents/save_load_testing/pandas_parquet.parquet'

# delete the file if it exists
if os.path.isfile(out_file):
    os.remove(out_file)

#write the file
df.to_parquet(out_file, engine='pyarrow')

import pyarrow.parquet as pq
t = pq.read_table(out_file)
cols = t.column_names

df_def = {}
for c in cols:
    df_def[c] = ak.array(t[c].to_numpy())
    
df = ak.DataFrame(df_def)
print(df.__repr__())

   Random_A  Random_B  Random_C
0         2         2         1
1         1         1         4
2         3         4         3
3         0         4         4
4         2         3         3 (5 rows x 3 columns)


## Export Parquet to Pandas
I believe we can just write this directly to pandas

In [22]:
#df.save_table(out_file, file_format='Parquet')

pd_read = '/Users/ethandebandi/Documents/save_load_testing/pandas_parquet_LOCALE0000.parquet'
pd.read_parquet(pd_read)

,Random_C,Random_B,Random_A
0,1,2,2
1,4,1,1
2,3,4,3
3,4,4,0
4,3,3,2


# Additional Notes
- Need to be able to handle import/export pdarray, dataframe
    - Other types?
- parameter to tell type we are loading
    - infer -> Multiple columns goes to DataFrame. Single goes to pdarray.
    -pdarray
    -dataframe
- Should we use a parameter for `import_from` and `export_to` that will indicate any specifics that we need to handle with how different formats configure the files.

In [10]:
ak.disconnect()

disconnected from arkouda server tcp://*:5555
